In [2]:
from ultralytics import YOLO
from PIL import Image
import pandas as pd
import numpy as np

In [8]:
# prior knowledge of dimension of furniture:
dimensions = {'couch':[2.5,.75],
              'chair':[1,1],
              'vase':[0.3,1.2],
              'bowl':[0.1,0.1]}

In [11]:
class AreaEstimator:
    def __init__(self, image_path, dimensions, conf_thres = 0.5, given_length = None, given_ref = None):
        self.image_path = image_path
        self.conf_thres = conf_thres
        self.dimensions = dimensions
        if given_length and given_ref:
            self.given_length = given_length
            self.given_ref = given_ref
        self.model = YOLO('yolov8m.pt')
        self.result = self.model.predict(image_path)
    
    def est_area(self, target_area):
        detected_objects = pd.DataFrame({
            'Category': self.result[0].boxes.cls.tolist(),
            'Confidence': self.result[0].boxes.conf.tolist(),
            'Coordinates': self.result[0].boxes.xyxy.tolist()
        })
        detected_objects['Category'] = detected_objects['Category'].map(self.result[0].names)

        prob_ratio = []

        for index, row in detected_objects.iterrows():
            if row['Confidence'] > self.conf_thres and row['Category'] in self.dimensions:
                hori_ratio = self.dimensions[row['Category']][0]/(row['Coordinates'][2]-row['Coordinates'][0])
                vert_ratio = self.dimensions[row['Category']][1]/(row['Coordinates'][3]-row['Coordinates'][1])

                prob_ratio.append(hori_ratio)
                prob_ratio.append(vert_ratio)

        ratio = np.mean(prob_ratio)
        est = (target_area[2]-target_area[0]) * ratio * (target_area[3]-target_area[1]) * ratio
        #print(prob_ratio)
        #print(ratio)
        print(f'The selcted area is {est} m2')
        
    
    def est_with_given(self, target_area):
        assert self.given_length is not None and self.given_ref is not None, "Given refer length must not be None"
        ratio = self.given_length/abs(self.given_ref[1]-self.given_ref[0])
        est = (target_area[2]-target_area[0]) * ratio * (target_area[3]-target_area[1]) * ratio
        #print(ratio)
        print(f'The selcted area is {est} m2')


In [12]:
#cropped pic is (1093,629), original is (4800,2754) 

target_pic = AreaEstimator("./livingroom_unmarked.jpg", dimensions, given_length=1.5, given_ref=[1258*1093/4800, 2291*629/2754])


target_pic.est_area([775*1093/4800, 568*629/2764, 1775*1093/4800, 1192*629/2764])


image 1/1 c:\Users\Teddywolf\iCloudDrive\\Synergy\Synergy\livingroom_unmarked.jpg: 384x640 2 bowls, 1 chair, 2 couchs, 3 potted plants, 3 vases, 737.0ms
Speed: 3.0ms preprocess, 737.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
The selcted area is 1.1546018661533597 m2


In [13]:
target_pic.est_with_given([775*1093/4800, 568*629/2764, 1775*1093/4800, 1192*629/2764])

The selcted area is 1.2975075807757552 m2
